## 初期設定

In [1]:
from mymodule import MLs
from mymodule import myfunc as mf
import numpy as np
import pandas as pd
import glob
import os
from dotenv import load_dotenv
load_dotenv()
# root_dir = os.environ["root_dir"]
root_dir = "../"

データの参照先の設定、インスタンス作成

In [4]:
TARGET = "mag_tupledxy" #ALTIMAGES等のディレクトリにくっつけます
#それぞれ1番最後の'/'は忘れずつけること
ALTIMAGES0 = root_dir + "imgout/0117_not/"#リコネクションなし分割済データフォルダ.makesepnpy.ipynbであらかじめ分割データを入れておく
ALTIMAGES1 = root_dir + "imgout/0117_1X/"#リコネクションあり分割済データフォルダ.makesepnpy.ipynbであらかじめ分割データを入れておく
LABEL_SOURCE0 = root_dir + "txt/labels0/"#リコネクションありのラベリングcsvのあるフォルダ
LABEL_SOURCE1 = root_dir + "txt/labels1X/"#リコネクションなしのラベリングcsvのあるフォルダ
# IMGSHAPE = [10,100]#出来れば画像サイズはすべて同じで合ってほしい。違うサイズが混じる場合は最も多いサイズを指定すること
IMGSHAPE = [10,100,2]
DO_PCA = False #データ圧縮。IMGSHAPEと、データサイズが途方もなく多ければTrueにする。
randomstate = None #学習時のパラメータ。初期値はNone。結果を固定したければ1とかに設定する。

ML = MLs.ML(TARGET, ALTIMAGES0, ALTIMAGES1, LABEL_SOURCE0, LABEL_SOURCE1, IMGSHAPE, DO_PCA,randomstate=randomstate)

ValueError: Too many dimensions: 3 > 2.

### 例

In [ ]:
#学習
#モジュールに書いてるコードで実行
# model = ML.linearSVC()   
# ML.kneighbors()
# ML.rbfSVC()
# ML.XGBoost()

#or
#自由に記述し、MLクラスを利用して実行
from sklearn.svm import LinearSVC
def mylinearSVC(self):    
        model = LinearSVC(C=0.3, random_state=self.randomstate) # インスタンスを生成
        model.fit(self.X_train, self.y_train) # モデルの学習
        # 学習データに対する精度
        print("Train :", model.score(self.X_train, self.y_train)) 
        # テストデータに対する精度
        print("Test :", model.score(self.X_test, self.y_test)) 
        print(model.predict(self.X_test))
        pred = model.predict(self.X_test)
        svmres = pd.DataFrame(np.array([self.path_test, self.y_test, pred]).T, columns=["path", "y", "predict"])
        # print(classification_report(self.y_test, pred))
        return model
svm = mylinearSVC(ML)

Train : 0.9398545076643284
Test : 0.9148807727135527
[0. 0. 0. ... 1. 1. 1.]


c:\Users\TsugiKawa\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
#参照可能なもの
ML.TARGET = TARGET#magfieldx, pressure,,,
ML.PATH1
ML.PATH0
ML.ALLTARINDATA0#データ参照先フォルダ
ML.ALLTARINDATA1
ML.PATH0TEST#テストデータ一覧
ML.PATH1TEST
ML.PATH0TRAIN#学習データ
ML.PATH1TRAIN
ML.X_test
ML.y_test
ML.path_test
ML.X_train
ML.y_train
ML.path_train
#後から変更可能なもの
ML.randomstate = 1

['../imgout/test1227/magfieldx/magfieldx_49.10.3_47.773.npy',
 '../imgout/test1227/magfieldx/magfieldx_49.10.6_47.773.npy',
 '../imgout/test1227/magfieldx/magfieldx_49.12.13_147.452.npy',
 '../imgout/test1227/magfieldx/magfieldx_49.13.13_42.607.npy',
 '../imgout/test1227/magfieldx/magfieldx_49.7.13_127.625.npy',
 '../imgout/test1227/magfieldx/magfieldx_49.10.4_47.773.npy',
 '../imgout/test1227/magfieldx/magfieldx_49.13.13_102.420.npy',
 '../imgout/test1227/magfieldx/magfieldx_49.10.3_82.620.npy']

In [ ]:
#学習モデルをファイルに保存、読み込み例
import pickle
path = "pickletest"

file = open(path, "wb") 
pickle.dump(svm, file)
file.close

with open(path, 'rb') as f:
    dict = pickle.load(f)
print(svm)

LinearSVC(C=0.3)


## 以下自由に

In [ ]:
model = ML.XGBoost()  

Train : 0.9988993710691824
Test : 0.9974433893352812
[1 0 0 ... 1 1 1]
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00       735
         1.0       1.00      1.00      1.00      2003

    accuracy                           1.00      2738
   macro avg       1.00      1.00      1.00      2738
weighted avg       1.00      1.00      1.00      2738



In [2]:
import pickle
picout = root_dir + "MLres/"

for target in ["density", "enstrophy", "pressure","magfieldx", "magfieldy", "velocityx","velocityy"]:
    for tag, altimages1, labelsources1 in [("X","imgout/0117_1X/","txt/labels1X/"), ("O","imgout/0117_1O/", "txt/labels1O/")]:
        TARGET = target
        #それぞれ1番最後の'/'は忘れずつけること
        ALTIMAGES0 = root_dir + "imgout/0117_not/"#リコネクションなし分割済データフォルダ
        ALTIMAGES1 = root_dir + altimages1#リコネクションあり分割済データフォルダ
        LABEL_SOURCE0 = root_dir + "txt/labels0/"#リコネクションありのラベリングcsvのあるフォルダ
        LABEL_SOURCE1 = root_dir + labelsources1#リコネクションなしのラベリングcsvのあるフォルダ
        IMGSHAPE = [10,100]#出来れば画像サイズはすべて同じで合ってほしい。違うサイズが混じる場合は最も多いサイズを指定すること
        DO_PCA = False #データ圧縮。IMGSHAPEと、データサイズが途方もなく多ければTrueにする。
        randomstate = None #学習時のパラメータ。初期値はNone。結果を固定したければ1とかに設定する。

        ML = MLs.ML(TARGET, ALTIMAGES0, ALTIMAGES1, LABEL_SOURCE0, LABEL_SOURCE1, IMGSHAPE, DO_PCA,randomstate=randomstate)
        def savemodel(model, methodname):
            picout = root_dir + "MLres/"
            path = picout + "model_" + target + "_" + tag + "_" + methodname
            file = open(path, "wb") 
            pickle.dump(model, file)
            file.close()
            print("model_" + target + "_" + tag + "_" + methodname+"\n")
            print("------------------\n\n\n\n")
        model = ML.linearSVC()   
        savemodel(model, "linearSVC")
        model = ML.kneighbors()
        savemodel(model, "kneighbors")
        model = ML.rbfSVC()
        savemodel(model, "rbfSVC")
        model = ML.XGBoost()
        savemodel(model, "XGBoost")

PCA 非実施


c:\Users\TsugiKawa\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Train : 0.8829566121070408
Test : 0.86296408089345
[0. 0. 1. ... 1. 1. 1.]
              precision    recall  f1-score   support

         0.0       0.83      0.67      0.74       967
         1.0       0.87      0.94      0.91      2346

    accuracy                           0.86      3313
   macro avg       0.85      0.81      0.82      3313
weighted avg       0.86      0.86      0.86      3313

model_density_X_linearSVC

------------------




Train : 0.9506365289685632
Test : 0.9441593721702385
[0. 0. 0. ... 1. 1. 1.]
              precision    recall  f1-score   support

         0.0       0.94      0.86      0.90       967
         1.0       0.94      0.98      0.96      2346

    accuracy                           0.94      3313
   macro avg       0.94      0.92      0.93      3313
weighted avg       0.94      0.94      0.94      3313

model_density_X_kneighbors

------------------




Train : 0.9580410496232787
Test : 0.9387262300030184
[0. 0. 0. ... 1. 1. 1.]
              pr

c:\Users\TsugiKawa\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Train : 0.9350553505535055
Test : 0.9179539993134226
[0. 1. 0. ... 1. 1. 0.]
              precision    recall  f1-score   support

         0.0       0.87      0.89      0.88       967
         1.0       0.94      0.93      0.94      1946

    accuracy                           0.92      2913
   macro avg       0.91      0.91      0.91      2913
weighted avg       0.92      0.92      0.92      2913

model_density_O_linearSVC

------------------




Train : 0.9176383763837639
Test : 0.9196704428424305
[0. 0. 0. ... 1. 1. 1.]
              precision    recall  f1-score   support

         0.0       0.84      0.93      0.88       967
         1.0       0.96      0.91      0.94      1946

    accuracy                           0.92      2913
   macro avg       0.90      0.92      0.91      2913
weighted avg       0.92      0.92      0.92      2913

model_density_O_kneighbors

------------------




Train : 0.9174907749077491
Test : 0.9117748026089941
[0. 0. 0. ... 1. 1. 0.]
              

...

Train : 0.8822140221402214
Test : 0.8513559903879162
[0. 0. 1. ... 1. 1. 1.]
              precision    recall  f1-score   support

         0.0       0.79      0.75      0.77       967
         1.0       0.88      0.90      0.89      1946

    accuracy                           0.85      2913
   macro avg       0.84      0.83      0.83      2913
weighted avg       0.85      0.85      0.85      2913

model_velocityx_O_linearSVC

------------------




Train : 0.8519557195571956
Test : 0.8389975969790594
[1. 0. 0. ... 1. 1. 1.]
              precision    recall  f1-score   support

         0.0       0.75      0.77      0.76       967
         1.0       0.89      0.87      0.88      1946

    accuracy                           0.84      2913
   macro avg       0.82      0.82      0.82      2913
weighted avg       0.84      0.84      0.84      2913

model_velocityx_O_kneighbors

------------------




Train : 0.8977121771217712
Test : 0.8637143837967731
[1. 1. 1. ... 1. 1. 1.]
          

c:\Users\TsugiKawa\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Train : 0.9676539360872954
Test : 0.9468759432538485
[0. 1. 0. ... 1. 1. 1.]
              precision    recall  f1-score   support

         0.0       0.91      0.91      0.91       967
         1.0       0.96      0.96      0.96      2346

    accuracy                           0.95      3313
   macro avg       0.94      0.94      0.94      3313
weighted avg       0.95      0.95      0.95      3313

model_velocityy_X_linearSVC

------------------




Train : 0.89893478825669
Test : 0.8934500452761848
[0. 0. 0. ... 1. 1. 1.]
              precision    recall  f1-score   support

         0.0       0.81      0.83      0.82       967
         1.0       0.93      0.92      0.92      2346

    accuracy                           0.89      3313
   macro avg       0.87      0.88      0.87      3313
weighted avg       0.89      0.89      0.89      3313

model_velocityy_X_kneighbors

------------------




Train : 0.9612886464016628
Test : 0.9444612134017507
[0. 0. 0. ... 1. 1. 1.]
            

c:\Users\TsugiKawa\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Train : 0.9354981549815499
Test : 0.9224167524888431
[0. 0. 0. ... 1. 1. 1.]
              precision    recall  f1-score   support

         0.0       0.90      0.86      0.88       967
         1.0       0.93      0.95      0.94      1946

    accuracy                           0.92      2913
   macro avg       0.92      0.91      0.91      2913
weighted avg       0.92      0.92      0.92      2913

model_velocityy_O_linearSVC

------------------




Train : 0.9078966789667897
Test : 0.9069687607277721
[0. 0. 0. ... 1. 1. 1.]
              precision    recall  f1-score   support

         0.0       0.89      0.83      0.85       967
         1.0       0.92      0.95      0.93      1946

    accuracy                           0.91      2913
   macro avg       0.90      0.89      0.89      2913
weighted avg       0.91      0.91      0.91      2913

model_velocityy_O_kneighbors

------------------




Train : 0.9030258302583026
Test : 0.891520768966701
[0. 1. 0. ... 1. 1. 1.]
           

In [ ]:
def loadmodel(modelsdir, target, tag, methodname):
    path = picout + "model_" + target + "_" + tag + "_" + methodname
    with open(path, 'rb') as f:
        return pickle.load(f)


In [ ]:
modelspath = glob.glob(picout)
    

In [5]:
import pickle
picout = root_dir + "MLres/"

for target in ["magxy"]:
    for tag, altimages1, labelsources1 in [("X","imgout/0117_1X/","txt/labels1X/"), ("O","imgout/0117_1O/", "txt/labels1O/")]:
        TARGET = target
        #それぞれ1番最後の'/'は忘れずつけること
        ALTIMAGES0 = root_dir + "imgout/0117_not/"#リコネクションなし分割済データフォルダ
        ALTIMAGES1 = root_dir + altimages1#リコネクションあり分割済データフォルダ
        LABEL_SOURCE0 = root_dir + "txt/labels0/"#リコネクションありのラベリングcsvのあるフォルダ
        LABEL_SOURCE1 = root_dir + labelsources1#リコネクションなしのラベリングcsvのあるフォルダ
        IMGSHAPE = [10,100]#出来れば画像サイズはすべて同じで合ってほしい。違うサイズが混じる場合は最も多いサイズを指定すること
        DO_PCA = False #データ圧縮。IMGSHAPEと、データサイズが途方もなく多ければTrueにする。
        randomstate = None #学習時のパラメータ。初期値はNone。結果を固定したければ1とかに設定する。

        ML = MLs.ML(TARGET, ALTIMAGES0, ALTIMAGES1, LABEL_SOURCE0, LABEL_SOURCE1, IMGSHAPE, DO_PCA,randomstate=randomstate)
        def savemodel(model, methodname):
            picout = root_dir + "MLres/"
            path = picout + "model_" + target + "_" + tag + "_" + methodname
            file = open(path, "wb") 
            pickle.dump(model, file)
            file.close()
            print("model_" + target + "_" + tag + "_" + methodname+"\n")
            print("------------------\n\n\n\n")
        model = ML.linearSVC()   
        savemodel(model, "linearSVC")
        model = ML.kneighbors()
        savemodel(model, "kneighbors")
        model = ML.rbfSVC()
        savemodel(model, "rbfSVC")
        model = ML.XGBoost()
        savemodel(model, "XGBoost")

PCA 非実施


c:\Users\TsugiKawa\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Train : 0.9340088334632372
Test : 0.9194083911862361
[1. 0. 1. ... 1. 1. 1.]
              precision    recall  f1-score   support

         0.0       0.90      0.82      0.86       967
         1.0       0.93      0.96      0.94      2346

    accuracy                           0.92      3313
   macro avg       0.91      0.89      0.90      3313
weighted avg       0.92      0.92      0.92      3313

model_magxy_X_linearSVC

------------------




Train : 0.9172512340867758
Test : 0.9136734077875037
[1. 0. 1. ... 1. 1. 1.]
              precision    recall  f1-score   support

         0.0       0.98      0.72      0.83       967
         1.0       0.90      0.99      0.94      2346

    accuracy                           0.91      3313
   macro avg       0.94      0.86      0.89      3313
weighted avg       0.92      0.91      0.91      3313

model_magxy_X_kneighbors

------------------




Train : 0.8777604572616264
Test : 0.8744340476909146
[1. 0. 1. ... 1. 1. 1.]
              prec

c:\Users\TsugiKawa\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Train : 0.9551291512915129
Test : 0.9364915894267078
[0. 0. 0. ... 1. 1. 1.]
              precision    recall  f1-score   support

         0.0       0.94      0.87      0.90       967
         1.0       0.94      0.97      0.95      1946

    accuracy                           0.94      2913
   macro avg       0.94      0.92      0.93      2913
weighted avg       0.94      0.94      0.94      2913

model_magxy_O_linearSVC

------------------




Train : 0.8977121771217712
Test : 0.8976999656711294
[0. 1. 0. ... 1. 1. 1.]
              precision    recall  f1-score   support

         0.0       0.99      0.70      0.82       967
         1.0       0.87      1.00      0.93      1946

    accuracy                           0.90      2913
   macro avg       0.93      0.85      0.87      2913
weighted avg       0.91      0.90      0.89      2913

model_magxy_O_kneighbors

------------------




Train : 0.8922509225092251
Test : 0.8870580157912805
[0. 1. 0. ... 1. 1. 1.]
              prec